In [1]:
!pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers==3.4.0
!pip install nltk spacy==2.1.6
!python -m spacy download en

# for torch-geometric
!pip install torch-scatter==2.0.7 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-sparse==0.6.9 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-geometric==1.7.0 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.0+cu101 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.0+cu101 which is incompatible.
     |████████████████████████████████| 1.3 MB 5.0 MB/s 
     |████████████████████████████████| 2.9 MB 61.7 MB/s 
     |████████████████████████████████| 1.2 MB 57.9 MB/s 
     |████████████████████████████████| 895 kB 66.7 MB/s

In [ ]:
!git clone https://github.com/michiyasunaga/qagnn.git

Cloning into 'qagnn'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 85 (delta 33), reused 64 (delta 18), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [ ]:
%cd qagnn

/content/qagnn


In [ ]:
!chmod 755 ./download_raw_data.sh
!./download_raw_data.sh
!python preprocess.py -p <num_processes>

In [ ]:
!chmod 755 ./download_preprocessed_data.sh
!./download_preprocessed_data.sh

--2022-01-25 16:32:53--  https://nlp.stanford.edu/projects/myasu/QAGNN/data_preprocessed_release.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4226838278 (3.9G) [application/zip]
Saving to: ‘data_preprocessed_release.zip’

data_preprocessed_r 100%[===================>]   3.94G  6.58MB/s    in 8m 12s  

2022-01-25 16:41:06 (8.19 MB/s) - ‘data_preprocessed_release.zip’ saved [4226838278/4226838278]

Archive:  data_preprocessed_release.zip
   creating: data_preprocessed_release/
   creating: data_preprocessed_release/cpnet/
  inflating: data_preprocessed_release/cpnet/conceptnet.en.unpruned.graph  
  inflating: data_preprocessed_release/cpnet/conceptnet.en.csv  
  inflating: data_preprocessed_release/cpnet/matcher_patterns.json  
  inflating: data_preprocessed_release/cpnet/conceptnet-assertions-5.6.0.csv  
  inflating: data_preproc

In [ ]:
!chmod 755 ./run_qagnn__csqa.sh
!./run_qagnn__csqa.sh

***** hyperparameters *****
dataset: csqa
enc_name: roberta-large
batch_size: 64
learning_rate: elr 1e-5 dlr 1e-3
gnn: dim 200 layer 5
******************************
tcmalloc: large alloc 3273826304 bytes == 0x55b86bac4000 @  0x7f9bf6ccc1e7 0x7f9ba259346e 0x7f9ba25e7e7c 0x7f9ba25e8aaf 0x7f9ba268a470 0x55b860cef544 0x55b860cef240 0x55b860d63627 0x55b860d5d9ee 0x55b860cf0bda 0x55b860d5f737 0x55b860d5d9ee 0x55b860cf0bda 0x55b860d62d00 0x55b860d5d9ee 0x55b860cf0bda 0x55b860d5e915 0x55b860d5dced 0x55b860cf0bda 0x55b860d5e915 0x55b860cf0afa 0x55b860d5e915 0x55b860d5d9ee 0x55b860d5d6f3 0x55b860e274c2 0x55b860e2783d 0x55b860e276e6 0x55b860dff163 0x55b860dfee0c 0x7f9bf5ab6bf7 0x55b860dfecea
tcmalloc: large alloc 3273826304 bytes == 0x55b92ecee000 @  0x7f9bf6ccc1e7 0x7f9ba259346e 0x7f9ba25e3c7b 0x7f9ba25e3d18 0x7f9ba268b010 0x7f9ba268b73c 0x7f9ba268b85d 0x55b860cf1749 0x7f9ba25d0ef7 0x55b860cef437 0x55b860cef240 0x55b860d62973 0x55b860d5d9ee 0x55b860cf0bda 0x55b860d62d00 0x55b860d5dced 0x55b860c

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1

dataset="csqa"
model='roberta-large'
shift
shift
args=$@

elr="1e-5"
dlr="1e-3"
bs=64
mbs=2
n_epochs=15
num_relation=38 #(17 +2) * 2: originally 17, add 2 relation types (QA context -> Q node; QA context -> A node), and double because we add reverse edges

k=5 #num of gnn layers
gnndim=200

save_dir_pref='saved_models'
mkdir -p $save_dir_pref
mkdir -p logs

for seed in 0; do
  python3 -u qagnn.py --dataset $dataset \
      --encoder $model -k $k --gnn_dim $gnndim -elr $elr -dlr $dlr -bs $bs -mbs $mbs --fp16 true --seed $seed \
      --num_relation $num_relation \
      --n_epochs $n_epochs --max_epochs_before_stop 10  \
      --train_adj data/${dataset}/graph/train.graph.adj.pk \
      --dev_adj   data/${dataset}/graph/dev.graph.adj.pk \
      --test_adj  data/${dataset}/graph/test.graph.adj.pk \
      --train_statements  data/${dataset}/statement/train.statement.jsonl \
      --dev_statements  data/${dataset}/statement/dev.statement.jsonl \
      --test_statements  data/${dataset}/statement/test.statement.jsonl \
      --save_model \
      --save_dir ${save_dir_pref}/${dataset}/enc-${model}__k${k}__gnndim${gnndim}__bs${bs}__seed${seed}__${dt} $args \
  > logs/train_${dataset}__enc-${model}__k${k}__gnndim${gnndim}__bs${bs}__seed${seed}__${dt}.log.txt
done

SyntaxError: ignored

In [ ]:
!chmod 755 ./eval_qagnn__csqa.sh
!./eval_qagnn__csqa.sh

******************************
dataset: csqa
******************************
ecf67aaf4725
pid: 1029
screen: 

gpu: 0

tcmalloc: large alloc 3273826304 bytes == 0x55b91b926000 @  0x7fb771bec1e7 0x7fb71d4b346e 0x7fb71d507e7c 0x7fb71d508aaf 0x7fb71d5aa470 0x55b90fc28544 0x55b90fc28240 0x55b90fc9c627 0x55b90fc969ee 0x55b90fc29bda 0x55b90fc98737 0x55b90fc969ee 0x55b90fc29bda 0x55b90fc9bd00 0x55b90fc969ee 0x55b90fc29bda 0x55b90fc97915 0x55b90fc29afa 0x55b90fc97915 0x55b90fc29afa 0x55b90fc97915 0x55b90fc969ee 0x55b90fc966f3 0x55b90fd604c2 0x55b90fd6083d 0x55b90fd606e6 0x55b90fd38163 0x55b90fd37e0c 0x7fb7709d6bf7 0x55b90fd37cea
tcmalloc: large alloc 3273826304 bytes == 0x55b9deb50000 @  0x7fb771bec1e7 0x7fb71d4b346e 0x7fb71d503c7b 0x7fb71d503d18 0x7fb71d5ab010 0x7fb71d5ab73c 0x7fb71d5ab85d 0x55b90fc2a749 0x7fb71d4f0ef7 0x55b90fc28437 0x55b90fc28240 0x55b90fc9b973 0x55b90fc969ee 0x55b90fc29bda 0x55b90fc9bd00 0x55b90fc29afa 0x55b90fc97915 0x55b90fc29afa 0x55b90fc97915 0x55b90fc969ee 0x55b90fc966f